In [16]:
import torch
import random
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
from torchvision import transforms

from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.datasets import CIFAR10
from avalanche.benchmarks.utils import AvalancheDataset
from avalanche.models import SLDAResNetModel
from avalanche.training import StreamingLDA
from avalanche.logging import InteractiveLogger
from avalanche.logging import InteractiveLogger, WandBLogger
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.training.plugins import EvaluationPlugin
from avalanche.evaluation.metrics import ExperienceAccuracy, ExperienceLoss, ExperienceForgetting, ExperienceCPUUsage, ExperienceMaxGPU, ExperienceMaxRAM, ExperienceTime, EpochAccuracy

In [17]:
seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
# torch.cuda.manual_seed_all(seed) # if use multi-GPU
cudnn.deterministic = True  # 연산 처리 속도 감소 -> 모델과 코드를 배포해야 하는 연구 후반 단계에 사용
cudnn.benchmark = False

In [18]:
train_set = CIFAR10('../data', train=True, download=True)
test_set = CIFAR10('../data', train=False, download=True)

train_transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

eval_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465),
                         (0.2023, 0.1994, 0.2010))
])

train_set = AvalancheDataset(train_set, transform=train_transform)
test_set = AvalancheDataset(test_set, transform=eval_transform)

Files already downloaded and verified
Files already downloaded and verified


In [19]:
interactive_logger = InteractiveLogger()
wandb_logger = WandBLogger(run_name="SLDA-CIFAR10")
eval_plugin = EvaluationPlugin(
    EpochAccuracy(),
    ExperienceAccuracy(),
    ExperienceLoss(),
    ExperienceForgetting(),
    ExperienceCPUUsage(),
    ExperienceMaxGPU(gpu_id=0),
    ExperienceMaxRAM(),
    ExperienceTime(),
    loggers=[interactive_logger, wandb_logger])

wandb: wandb version 0.12.17 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [20]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_class = 10
incremental = 5
fixed_class_order = [4, 1, 7, 5, 3, 9, 0, 8, 6, 2]

scenario = nc_benchmark(train_dataset=train_set,
                        test_dataset=test_set,
                        n_experiences=incremental,
                        task_labels=True,
                        seed=seed,
                        shuffle=False,
                        fixed_class_order=fixed_class_order,
                        )

model = SLDAResNetModel(imagenet_pretrained=False, device=device)

# Prepare for training & testing
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()

In [21]:
epoch = 70
train_batch = 128
eval_batch = 64

strategies = StreamingLDA(model, criterion, input_size=512, num_classes=num_class, train_epochs=epoch, train_mb_size=train_batch, eval_mb_size=eval_batch, device=device, evaluator=eval_plugin)

pllugins :  [<avalanche.training.plugins.evaluation.EvaluationPlugin object at 0x0000022C572BC2B0>]
 73%|███████▎  | 58/79 [04:16<01:33,  4.43s/it]


In [22]:
for i, exp in enumerate(scenario.train_stream):
    eval_exps = [e for e in scenario.test_stream][: i + 1]
    strategies.train(exp)
    strategies.eval(eval_exps)

-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
100%|██████████| 79/79 [00:05<00:00, 13.54it/s]
Epoch 0 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8212
100%|██████████| 79/79 [00:05<00:00, 13.65it/s]
Epoch 1 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8607
100%|██████████| 79/79 [00:05<00:00, 13.61it/s]
Epoch 2 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8608
100%|██████████| 79/79 [00:05<00:00, 13.63it/s]
Epoch 3 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8615
100%|██████████| 79/79 [00:05<00:00, 14.22it/s]
Epoch 4 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8667
100%|██████████| 79/79 [00:05<00:00, 14.39it/s]
Epoch 5 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8645
100%|██████████| 79/79 [00:05<00:00, 14.44it/s]
Epoch 6 ended.
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.8622
100%|██████████| 79/79 [00:05<00:00, 14.57it/s]
Ep

KeyboardInterrupt: 